In [4]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable

import torchvision

#### Define constants and dataset
- TODO All these definitions should be done using CL args
- Set important constant such as CUDA use and batch size

In [5]:
use_gpu = torch.cuda.is_available()
batch_size = 32
dataset = torchvision.datasets.MNIST

#### Create DataSet and DataLoader objects

In [6]:
train_data = dataset('/datasets', train=True, download=False)
test_data = dataset('/datasets', train=False, download=False)

In [7]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=4)

### TODO List
- TODO write routing algorithm
- TODO write capsule layer architecture
- TODO write capsule network architecture
- TEST Squash function
- TODO Review Squash

In [66]:
def _squash(tensor):
    '''
    TODO test
    TODO review input format.
    Squash function, defined in [1]. Works as a nonlinearity for CapsNets.
    Input tensor will be of format (bs, num_atoms, C, H, W) or (bs, num_atoms, C)
    Norm should be computed on the axis representing the number of atoms.
    params:
        tensor:    torch Variable containing n-dimensional tensor
    output:
        (||tensor||^2 / (1+ ||tensor||^2)) * tensor/||tensor||
    '''
    norm = torch.norm(tensor, p=2, dim=1, keepdim=True)
    sq_norm = norm ** 2 # Avoid computing square twice
        
    return tensor.div(norm) * sq_norm/(1 + sq_norm)

def _leaky_routing(logits):
    '''
    TODO write doc
    
    Parameters:
        logits: Tensor of shape (bs, input_dim, output_dim)
    
    '''
    leak = torch.zeros_like(logits)
    leak = leak.sum(dim=2, keepdim=True)
    leaky_logits = torch.cat([logits, leak], dim=2)
    leaky_routing = F.softmax(leaky_logits, dim=2)
    return leaky_routing[:, :, :-1, ...]

def _update_routing(votes, biases, logit_shape, num_dims, input_dim, output_dim, num_routing, leaky):
    '''
    TODO test
    TODO write doc
    Parameters:
        votes: Tensor (bs, input_dim, output_dim, output_atoms)
        biases: Tensor (output_dim, output_atoms)
        logit_shape: Tensor (bs, input_dim, output_dim)
        num_dims:
        input_dim: Integer.
        output_dim: Integer.
        num_routing: Integer. Number of routing iterations.
        leaky: Boolean. Whether to use leaky routing or not.
    '''

    votes_t_shape = [3, 0, 1, 2] # [output_atoms, bs, input_dim, output_dim, ...]
    for i in range(num_dims - 4):
        votes_t_shape += [i + 4]
    r_t_shape = [1, 2, 3, 0] # [bs, input_dim, output_dim, output_atoms, ...]
    for i in range(num_dims - 4):
        r_t_shape += [i + 4]
    votes_trans = votes.permute(*votes_t_shape)
    
    activations = None
    logits = Variable(torch.zeros(logit_shape))
    for i in range(num_routing):
        # route: [bs, input_dim, output_dim]
        if leaky:
            route = _leaky_routing(logits)
        else:
            route = F.softmax(logits, dim=2)
        
        preactivate_unrolled = route * votes_trans
        preact_trans = preactivate_unrolled.permute(*r_t_shape)
        preactivate = torch.sum(preact_trans, dim=1) + biases
        activation = _squash(preactivate)
        if activations is None:
            activations = torch.unsqueeze(activation, dim=-1)
        else:
            activations = torch.cat([activations, torch.unsqueeze(activation, dim=-1)], dim=-1)            
        #distances: [bs, input_dim, output_dim]
        act_3d = torch.unsqueeze(activation, dim=1)
        tile_shape = np.ones(num_dims, dtype=np.int32).tolist()
        tile_shape[1] = input_dim
        act_replicated = act_3d.repeat(*tile_shape)
        distances = torch.sum(votes * act_replicated, dim=3)
        logits += distances
    return activations[..., -1]

In [69]:
votes = Variable(torch.Tensor(np.reshape(np.arange(8, dtype=np.float32), (1, 2, 2, 2))))
biases = Variable(torch.zeros((2, 2)))
logit_shape = (1, 2, 2)
_update_routing(
    votes,
    biases, 
    logit_shape, 
    num_dims=4,
    input_dim=2,
    output_dim=2,
    num_routing=1,
    leaky=False
    )

torch.Size([1, 2, 2])


Variable containing:
(0 ,.,.) = 
  0.4259  0.4998
  0.8518  0.8330
[torch.FloatTensor of size 1x2x2]

In [52]:
squash(Variable(torch.ones(10,10,10,10,10))).shape

NameError: name 'squash' is not defined

#### Optimizer definition according to default Tensorflow initiation
From Tensorflow [AdamOptimizer docs](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer):
```
__init__(
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam'
)```

These are also the default values for torch.optim.Adam

In [ ]:
optimizer = torch.optim.Adam()